In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

from collections import Counter
import string
import re
import argparse
import json
import sys

In [2]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)

In [3]:
def identify_question_type(question):
    if 'what' in question:
        return 'what'
    elif 'who' in question:
        return 'who'
    elif 'how' in question:
        return 'how'
    elif 'when' in question:
        return 'when'
    elif 'which' in question:
        return 'which'
    elif 'where' in question:
        return 'where'
    elif 'why' in question:
        return 'why'
    else:
        return 'other'

In [4]:
with open('../predictions.json') as f:
    predictions = json.load(f)
    
with open('../data/dev-v1.1.json') as f:
    dataset_json = json.load(f)
    dataset = dataset_json['data']

In [5]:
question_type_list = []
context_len_list = []
query_len_list = []
avg_answer_len_list = []
tmp_exact_match_list = []
tmp_f1_list = []

err_context = []
err_question = []
err_ground_truth = []
err_prediction = []
err_em = []


f1 = exact_match = total = 0
for article in dataset:
    for paragraph in article['paragraphs']:
        for qa in paragraph['qas']:
            total += 1
            if qa['id'] not in predictions:
                message = 'Unanswered question ' + qa['id'] + \
                          ' will receive score 0.'
                print(message, file=sys.stderr)
                continue
            ground_truths = list(map(lambda x: x['text'], qa['answers']))
            prediction = predictions[qa['id']]
            question = qa['question']
            
            question_type = identify_question_type(normalize_answer(question))
            context_len = len(normalize_answer(paragraph['context']).split())
            query_len = len(normalize_answer(qa['question']).split())
            avg_answer_len = sum([len(normalize_answer(item).split()) for item in ground_truths]) / len(ground_truths)
            
            tmp_exact_match = metric_max_over_ground_truths(
                exact_match_score, prediction, ground_truths)
            tmp_f1 = metric_max_over_ground_truths(
                f1_score, prediction, ground_truths)
            
            question_type_list.append(question_type)
            context_len_list.append(context_len)
            query_len_list.append(query_len)
            avg_answer_len_list.append(avg_answer_len)
            tmp_exact_match_list.append(tmp_exact_match)
            tmp_f1_list.append(tmp_f1)
            
            err_context.append(paragraph['context'])
            err_question.append(question)
            err_ground_truth.append(ground_truths)
            err_prediction.append(prediction)
            err_em.append(tmp_exact_match)
            
            exact_match += metric_max_over_ground_truths(
                exact_match_score, prediction, ground_truths)
            f1 += metric_max_over_ground_truths(
                f1_score, prediction, ground_truths)

exact_match = 100.0 * exact_match / total
f1 = 100.0 * f1 / total

In [6]:
df = pd.DataFrame({'context': err_context, 'question': err_question, 
                 'ground_truths': err_ground_truth, 'prediction': err_prediction, 
                 'em': err_em})

In [7]:
df = df[df['em'] == False].reset_index(drop=True)

In [8]:
df.shape

(3913, 5)

In [9]:
np.random.seed(27)
df = df.ix[np.random.choice(4336, 50)]

In [10]:
df.head()

,context,em,ground_truths,prediction,question
3768,The outcome was one of the most significant de...,False,[confirming Britain's position as the dominant...,anglo-french conflict,What was the significance of British win?
3912,"The pound-force has a metric counterpart, less...",False,"[sthène, sthène, sthène, sthène, sthène]",the metric slug,What is the seldom used force unit equal to on...
4237,NaN,NaN,NaN,NaN,NaN
2929,"In 1999, another special, Doctor Who and the C...",False,"[head writer and executive producer, Steven Mo...",executive producer,The script writer for the special went on to h...
1026,Some scholars have asserted that Luther taught...,False,"[antithetical, antithetical, antithetical]",antithetical in the sense that questions of fa...,What do scholars sometimes assert that Luther ...


In [11]:
for step, row in enumerate(df.iterrows()):
    print('Step: ')
    print(step+2)
    print('\n')
    temp = row[1]
    print('Context: ')
    print(temp['context'])
    print('\n')
    print('Question: ')
    print(temp['question'])
    print('\n')
    print('Ground Truths: ')
    print(temp['ground_truths'])
    print('\n')
    print('Prediction: ')
    print(temp['prediction'])
    print('\n')


Step: 
2


Context: 
The outcome was one of the most significant developments in a century of Anglo-French conflict. France ceded its territory east of the Mississippi to Great Britain. It ceded French Louisiana west of the Mississippi River (including New Orleans) to its ally Spain, in compensation for Spain's loss to Britain of Florida (Spain had ceded this to Britain in exchange for the return of Havana, Cuba). France's colonial presence north of the Caribbean was reduced to the islands of Saint Pierre and Miquelon, confirming Britain's position as the dominant colonial power in eastern North America.


Question: 
What was the significance of British win?


Ground Truths: 
["confirming Britain's position as the dominant colonial power in eastern North America", "confirming Britain's position as the dominant colonial power in eastern North America", "confirming Britain's position as the dominant colonial power in eastern North America", 'dominant colonial power', "confirming Britain'

In [ ]:
# manual category

In [28]:
manual_err_analysis = pd.read_excel('BCN_error_analysis.xlsx')

In [30]:
manual_err_analysis = manual_err_analysis.rename(columns={
    'imprecise': 'Imprecise answer boundaries',
    'syntactic': 'Syntactic complications and ambiguities',
    'paraphrase': 'Paraphrase problems',
    'mult--sentence': 'Multi-sentence',
    'padding sta': 'Padding strategy',
    'external': 'External knowledge'
})

In [31]:
manual_err_analysis = manual_err_analysis.fillna(0)

In [32]:
manual_err_analysis.to_excel('BCN_error_analysis.xlsx', index=False)

In [33]:
manual_err_analysis

,Imprecise answer boundaries,Syntactic complications and ambiguities,Paraphrase problems,Multi-sentence,Padding strategy,External knowledge
0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0
6,0.0,0.0,0.0,1.0,0.0,0.0
7,0.0,1.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,1.0,0.0,0.0
9,0.0,1.0,0.0,0.0,0.0,0.0


In [34]:
manual_err_analysis.sum()

Imprecise answer boundaries                23.0
Syntactic complications and ambiguities    15.0
Paraphrase problems                         2.0
Multi-sentence                              6.0
Padding strategy                            3.0
External knowledge                          1.0
dtype: float64

In [35]:
manual_err_analysis.sum() / 50

Imprecise answer boundaries                0.46
Syntactic complications and ambiguities    0.30
Paraphrase problems                        0.04
Multi-sentence                             0.12
Padding strategy                           0.06
External knowledge                         0.02
dtype: float64